In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

## Parameters

In [2]:
class Config:
    # Data loading params
    max_sentence_length = 40
    dev_sample_percentage = 0.1
    
    # Embeddings
    embedding_dim = 300
    
    # RNN
    hidden_size = 300
    
    # Training parameters
    batch_size = 40
    num_epochs = 20
    display_every = 200
    evaluate_every = 1000
    num_checkpoints = 5
    learning_rate = 0.001
    
    # Testing parameters
    checkpoint_dir = ''
    
    UNK = "$UNK$"
    NUM = "$NUM$"
    NONE = "O"
    PAD = '$PAD$'

## Dataset 

Load annotated corpus for named entity recognition

https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus

In [3]:
import nltk
import re
import os

class Dataset:
    def __init__(self):
        self.all_tags, self.all_words = [], [] 
        
    def processing_word(self, word):
        word = word.lower()
        if word.isdigit():
            word = Config.NUM
        return word
        
    def load_dataset(self, path):
        words_col, tags_col = [], []
        with open(path) as f:
            words, tags = [], []
            for line in f:
                line = line.strip()
                if (len(line) == 0 or line.startswith("-DOCSTART-")):
                    if len(words) != 0:
                        words_col.append(' '.join(words))
                        tags_col.append(' '.join(tags))
                        words, tags = [], []
                else:
                    ls = line.split(' ')
                    word, tag = ls[0],ls[3]
                    word = self.processing_word(word)
                    
                    words.append(word)
                    tags.append(tag)
                    
                    self.all_words.append(word)
                    self.all_tags.append(tag)
                    
        return pd.DataFrame({'words': words_col, 'tags': tags_col})
        
    def download_and_load_datasets(self):
        self.all_tags, self.all_words = [], [] 
        
        dataset = tf.keras.utils.get_file(
          fname="CoNLL-2003.zip", 
          origin="https://s3.ap-northeast-2.amazonaws.com/bowbowbow-storage/dataset/CoNLL-2003.zip", 
          extract=True)
        
        dir_path = os.path.join(os.path.dirname(dataset), 'CoNLL-2003')
        train_df = self.load_dataset(os.path.join(dir_path, 'eng.train'))
        test_df = self.load_dataset(os.path.join(dir_path, 'eng.testb'))
        return train_df, test_df

dataset = Dataset()
train_df, test_df = dataset.download_and_load_datasets()
train_df.head(10)

,tags,words
0,I-ORG O I-MISC O O O I-MISC O O,eu rejects german call to boycott british lamb .
1,I-PER I-PER,peter blackburn
2,I-LOC O,brussels 1996-08-22
3,O I-ORG I-ORG O O O O O O I-MISC O O O O O I-M...,the european commission said on thursday it di...
4,I-LOC O O O O I-ORG I-ORG O O O I-PER I-PER O ...,germany 's representative to the european unio...
5,O O O O O O O O O O O O O O O O O O O O I-ORG ...,""" we do n't support any such recommendation be..."
6,O O O O O O O O O O O O O O O O O O O O O O I-...,he said further scientific study was required ...
7,O O O O O O O I-ORG O O I-PER I-PER O O O O O ...,he said a proposal last month by eu farm commi...
8,I-PER O I-MISC O O O O I-LOC O I-LOC O O O O O...,fischler proposed eu-wide measures after repor...
9,O I-PER O O O O O O O I-ORG O O O O O O O O O ...,but fischler agreed to review his proposal aft...


In [4]:
word_list = list(set(dataset.all_words)) + [Config.PAD, Config.UNK]
word2idx = {w: i for i, w in enumerate(word_list)}
idx2word = {i: w for i, w in enumerate(word_list)}

tag_list = list(set(dataset.all_tags))
tag2idx = {w: i for i, w in enumerate(tag_list)}
idx2tag = {i: w for i, w in enumerate(tag_list)}

x, lengths, y = [], [], []

for index, row in train_df.iterrows():
    sentence = row['words'].split(' ')
    tags = row['tags'].split(' ')
    
    sentence = sentence[:Config.max_sentence_length]
    tags = tags[:Config.max_sentence_length]
    
    lengths.append(Config.max_sentence_length)
    x_row, y_row = [], []
    for word in sentence:
        x_row.append(word2idx[word])
    for tag in tags:
        y_row.append(tag2idx[tag])
    
    if len(sentence) < Config.max_sentence_length:
        lengths[-1] = len(sentence)
        x_row += [word2idx[Config.PAD]]* (Config.max_sentence_length - len(sentence))
        y_row += [tag2idx[Config.NONE]]* (Config.max_sentence_length - len(sentence))
        
    x.append(x_row)
    y.append(y_row)

In [5]:
x = np.array(x)
y = np.array(y)
lengths = np.array(lengths)

# Randomly shuffle data to split into train and dev
np.random.seed(11)
shuffle_indices = np.random.permutation(np.arange(len(y)))

x_shuffled, lengths_shuffled, y_shuffled = x[shuffle_indices], lengths[shuffle_indices], y[shuffle_indices]

# Split train/dev set
dev_sample_index = -1 * int(Config.dev_sample_percentage*float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:] 
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:] 
lengths_train, lengths_dev = lengths_shuffled[:dev_sample_index], lengths_shuffled[dev_sample_index:]

print("Train/Dev split: {:d}/{:d}\n".format(len(y_train), len(y_dev)))

Train/Dev split: 12637/1404



## Model

In [6]:
class Model:
    def __init__(self, 
               sequence_length, 
               num_classes, 
               vocab_size, 
               embedding_size, 
               hidden_size):
        
        self.input_x = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_x')
        self.input_y = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_y')
        self.sequence_lengths = tf.placeholder(tf.int32, shape=[None], name="sequence_lengths")
        
        initializer = tf.contrib.layers.xavier_initializer()
        
        # Embedding layer
        with tf.device('/cpu:0'), tf.variable_scope('text-embedding'):
            self.W_text = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -0.25, 0.25), name='W_text')
            self.embedded_chars = tf.nn.embedding_lookup(self.W_text, self.input_x) # [batch_size, sequence_length, embedding_size]
            
        # Bidirectional LSTM
        with tf.variable_scope("bi-lstm"):
            fw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
            bw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
            self.rnn_outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_cell,
                                                                  cell_bw=bw_cell,
                                                                  inputs=self.embedded_chars,
                                                                  sequence_length= self._length(self.input_x), # [batch_size],
                                                                  dtype=tf.float32)
            
            self.rnn_outputs = tf.concat([self.rnn_outputs[0], self.rnn_outputs[1]], axis=2) # [batch_size, sequence_length, 2*hidden_size]
            
        # Fully connected layer
        with tf.variable_scope('output'):
            self.W_output = tf.get_variable('W_output', shape=[2*hidden_size, num_classes],  dtype=tf.float32)
            self.b_output = tf.get_variable('b_output', shape=[num_classes], dtype=tf.float32, initializer=tf.zeros_initializer())
            
            rnn_outputs_flat = tf.reshape(self.rnn_outputs, [-1, 2*hidden_size])
            pred = tf.matmul(rnn_outputs_flat, self.W_output) + self.b_output
            
            self.logits = tf.reshape(pred, [-1, sequence_length, num_classes]) # [batch_size, sequence_length, num_classes]
            
        # Calculate mean corss-entropy loss
        with tf.variable_scope('loss'):
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            mask = tf.sequence_mask(self.sequence_lengths)
            losses = tf.boolean_mask(losses, mask)
            
            self.loss = tf.reduce_mean(losses) 
        
        # Accuracy    
        with tf.name_scope('accuracy'):
            self.predictions = tf.argmax(self.logits, 2, name='predictions')
            correct_predictions = tf.equal(self.predictions, tf.cast(self.input_y, tf.int64))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')
    
    # Length of the sequence data
    @staticmethod
    def _length(seq):
        relevant = tf.sign(tf.abs(seq))
        length = tf.reduce_sum(relevant, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length

## Function for train

In [7]:
# Pre-trained glove
def load_glove(embedding_dim, word2idx):
    download_path = tf.keras.utils.get_file(
      fname="glove.6B.zip", 
      origin="http://nlp.stanford.edu/data/glove.6B.zip", 
      extract=True)
    
    embedding_path = os.path.join(os.path.dirname(download_path), 'glove.6B.300d.txt')
    print('embedding_path :', embedding_path)

    # initial matrix with random uniform
    initW = np.random.randn(len(word2idx), embedding_dim).astype(np.float32) / np.sqrt(len(word2idx))
    # load any vectors from the glove
    print("Load glove file {0}".format(embedding_path))
    f = open(embedding_path, 'r', encoding='utf8')
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        if word in word2idx:
            initW[word2idx[word]] = embedding
    return initW

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            

## Train

In [8]:
import datetime
import time

from sklearn.metrics import f1_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():
    model = Model(
        sequence_length=x_train.shape[1],
        num_classes=len(tag_list),
        vocab_size=len(word_list),
        embedding_size=Config.embedding_dim,
        hidden_size=Config.hidden_size
    )
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = tf.train.AdadeltaOptimizer(Config.learning_rate).minimize(model.loss, global_step=global_step)
    
    # Output directory for models and summary
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "30.runs", timestamp))
    print("Writing to {}\n".format(out_dir))

    sess.run(tf.global_variables_initializer())

#     pretrain_W = load_glove(Config.embedding_dim, word2idx)
#     sess.run(model.W_text.assign(pretrain_W))
#     print("Success to load pre-trained glove model!\n")
    
    # Generate batches
    batches = batch_iter(list(zip(x_train, lengths_train, y_train)), Config.batch_size, Config.num_epochs)
    
    for batch in batches:
        x_batch, lengths_train, y_batch = zip(*batch)

        # Train
        feed_dict = {
            model.input_x: x_batch,
            model.input_y: y_batch,
            model.sequence_lengths: lengths_train,
        }

        _, step, loss, accuracy = sess.run(
            [train_op, global_step, model.loss, model.accuracy], feed_dict)


        # Training log display
        if step % Config.display_every == 0:
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

        # Evaluation
        if step % Config.evaluate_every == 0:
            print("\nEvaluation:")
            feed_dict = {
                model.input_x: x_dev,
                model.input_y: y_dev,
                model.sequence_lengths: lengths_dev,
            }
            loss, accuracy, predictions = sess.run(
                [model.loss, model.accuracy, model.predictions], feed_dict)

            from sklearn.metrics import classification_report
            
            print(classification_report(
                [idx2tag[y] for y in np.array(y_dev).flatten()], 
                [idx2tag[y] for y in np.array(predictions).flatten()]))

            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:.4f}\n".format(time_str, step, loss, accuracy))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


/home/seungwon/project/tf-notes/venv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Writing to /home/seungwon/project/tf-notes/30.runs/1557932535

2019-05-16T00:02:23.380068: step 200, loss 2.06631, acc 0.0525
2019-05-16T00:02:30.399414: step 400, loss 2.05272, acc 0.17375
2019-05-16T00:02:37.419910: step 600, loss 1.91376, acc 0.721875
2019-05-16T00:02:44.428180: step 800, loss 1.8607, acc 0.861875
2019-05-16T00:02:51.423989: step 1000, loss 1.95301, acc 0.905625

Evaluation:
              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00         1
      B-MISC       0.00      0.25      0.00         4
       B-ORG       0.00      0.00      0.00         0
       I-LOC       0.02      0.01      0.01       828
      I-MISC       0.02      0.03      0.03       451
       I-ORG       0.07      0.04      0.05      1056
       I-PER       0.06      0.02      0.03      1045
           O       0.95      0.93      0.94     52775

   micro avg       0.87      0.87      0.87     56160
   macro avg       0.14      0.16      0.13     56160
weight

## Tensorboard

```
tensorboard --logdir=./30.runs --host 0.0.0.0
```